In [1]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
text = "Replace  by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [1]:
import torch
import torch.nn as nn
import json
from transformers import BertTokenizer, BertModel
from sklearn.neighbors import KNeighborsClassifier

In [3]:
import json
list_sent=[]
Y_train=[]
with open('train.json', 'r') as file:
    data = file.read()
    data = json.loads(data)
    for label in data.keys():
        for sentence in data[label]:
            list_sent.append(sentence)
            Y_train.append(label)
    


In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

In [3]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [33]:
X_train=[]
for sent in list_sent:
    encoded_input = tokenizer(sent, return_tensors='pt')
    output = model(**encoded_input)
    X_train.append(output[1][0].detach().numpy())

In [37]:
# Assuming you have X_train and Y_train variables defined
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, Y_train)


KNeighborsClassifier(n_neighbors=1)

In [42]:
phrase="The role of credit scores in lending decisions is significant."
encoded_phrase = tokenizer(phrase, return_tensors='pt')
phrase_embedding = model(**encoded_phrase)[1][0].detach().numpy()
predicted_label = knn.predict([phrase_embedding])
print(predicted_label)


['Finance']


Feature based BERT

In [9]:
from transformers import BertTokenizer, BertModel
from sklearn.neighbors import KNeighborsClassifier
import json
import csv

class BertClassifFeatures():
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertModel.from_pretrained("bert-base-uncased")
        self.knn = KNeighborsClassifier(n_neighbors=1)
    def train(self,train_file):
        list_sent=[]
        Y_train=[]
        with open(train_file, 'r') as file:
            data = file.read()
            data = json.loads(data)
            for label in data.keys():
                for sentence in data[label]:
                    list_sent.append(sentence)
                    Y_train.append(label)
        X_train=[]
        for sent in list_sent:
            encoded_input = self.tokenizer(sent, return_tensors='pt')
            output = self.model(**encoded_input)
            X_train.append(output[1][0].detach().numpy())
        self.knn.fit(X_train, Y_train)
    def predict(self,phrase):
        encoded_phrase = self.tokenizer(phrase, return_tensors='pt')
        phrase_embedding = self.model(**encoded_phrase)[1][0].detach().numpy()
        predicted_label = self.knn.predict([phrase_embedding])
        return predicted_label
    def create_csv(self,test_file,csv_file):
        csv_file = open(csv_file, 'w',newline='')
        writer = csv.writer(csv_file)
        writer.writerow(["ID","Label"])
        i=0
        with open(test_file, 'r') as file:
            for line in file:
                pred=self.predict(line)[0]
                writer.writerow([str(i), str(pred)])
                i+=1
        csv_file.close()

In [10]:
test_file='./data/test_shuffle.txt'
csv_file='./data/feature_based_bert2.csv'	

In [11]:
bert=BertClassifFeatures()
bert.train('./data/train.json')

In [12]:
bert.create_csv(test_file,csv_file)

In [10]:
bert.predict("The Supreme Court has ruled in favor of the plaintiff in the high-profile case.")[0]

'Politics'

Finetuned BERT

In [46]:
import torch
import torch.nn as nn
import json
from transformers import BertTokenizer, BertModel
import random
import csv

class BertClassifFinetuned():
    def __init__(self,train_file,validation_file):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertModel.from_pretrained("bert-base-uncased")
        self.linear = torch.nn.Linear(768,12,bias=True)
        self.softmax = torch.nn.Softmax(dim=1)
        self.train_file=train_file
        list_sent=[]
        list_lab=[]
        with open(train_file, 'r') as file:
            data = file.read()
            data = json.loads(data)
            for label in data.keys():
                for sentence in data[label]:
                    list_sent.append(sentence)
                    list_lab.append(label)
        self.list_lab=list_lab
        self.list_sent=list_sent
        Y_train=[]
        self.dic_lab={}
        for i in range(len(list_lab)):
            if list_lab[i] not in self.dic_lab:
                self.dic_lab[list_lab[i]]=len(self.dic_lab)
            Y_train.append(self.dic_lab[list_lab[i]])
        X_train=[]
        for sent in list_sent:
            encoded_input = self.tokenizer(sent, return_tensors='pt')
            output = self.model(**encoded_input)
            X_train.append(output[1][0].detach().numpy())
        self.X_train=X_train
        self.Y_train=Y_train
        self.sent_train=list_sent
        self.lab_train=list_lab

        list_sent_val=[]
        list_lab_val=[]
        with open(validation_file, 'r') as file:
            data = file.read()
            data = json.loads(data)
            for label in data.keys():
                for sentence in data[label]:
                    list_sent_val.append(sentence)
                    list_lab_val.append(label)
        self.lab_val=list_lab_val
        self.sent_val=list_sent_val
        Y_val=[]
        X_val=[]
        for i,sent in enumerate(list_sent_val):
            encoded_input = self.tokenizer(sent, return_tensors='pt')
            output = self.model(**encoded_input)
            X_val.append(output[1][0].detach().numpy())
            Y_val.append(self.dic_lab[list_lab_val[i]])
        self.X_val=X_val
        self.Y_val=Y_val
    def train(self,epochs=10000):
        X_train=torch.tensor(self.X_train)
        Y_train=torch.tensor(self.Y_train)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.SGD(self.linear.parameters(), lr=0.001)

        train_data = list(zip(X_train, Y_train))
        random.shuffle(train_data)
        X_train, Y_train = zip(*train_data)
        X_train = torch.stack(X_train)
        Y_train = torch.tensor(Y_train)

        for epoch in range(epochs):
            optimizer.zero_grad()
            outputs = self.linear(X_train)
            outputs = self.softmax(outputs)
            loss = criterion(outputs, Y_train)
            loss.backward()
            optimizer.step()
            if epoch % 1000 == 0:
                print ('Epoch [{}/{}], Loss: {:.4f}, Val score: {:.4f}'.format(epoch+1, epochs, loss.item(), self.validation_score()))
    def data_augmentation(self):
        list_sent = self.sent_train
        new_sent=[]
        list_lab = self.lab_train
        for phrase in list_sent:
            words=phrase.split(" ")
            i=random.randint(0,len(words)-1)
            words[i]="[MASK]"
            phrase=" ".join(words)
            new_sent.append(phrase)
        X_train=[]
        for sent in list_sent:
            encoded_input = self.tokenizer(sent, return_tensors='pt')
            output = self.model(**encoded_input)
            X_train.append(output[1][0].detach().numpy())
        self.X_train = self.X_train + X_train
        self.Y_train = self.Y_train + self.Y_train

    def get_new_train(self,conf=0.99,test_file='./data/test_shuffle.txt'):
        list_sent=[]
        list_lab=[]
        data=[]
        with open(test_file, 'r') as file:
            for line in file:
                pred=bert.predict(line)
                data.append([pred[1],pred[0],line])
        for x in data:
            if x[0]>conf and x[2] not in self.sent_train:
                list_sent.append(x[2])
                list_lab.append(x[1])
        X_train=[]
        Y_train=[]
        for i,sent in enumerate(list_sent):
            encoded_input = self.tokenizer(sent, return_tensors='pt')
            output = self.model(**encoded_input)
            X_train.append(output[1][0].detach().numpy())
            Y_train.append(self.dic_lab[list_lab[i]])
        self.X_train = self.X_train + X_train
        self.Y_train = self.Y_train + Y_train
        self.sent_train = self.sent_train + list_sent
        self.lab_train = self.lab_train + list_lab
    def save(self,path='model.pth'):
        torch.save(self.linear.state_dict(), path)
    def load(self,path='model.pth'):
        self.linear.load_state_dict(torch.load(path))
    def predict(self,phrase):
        encoded_phrase = self.tokenizer(phrase, return_tensors='pt')
        phrase_embedding = self.model(**encoded_phrase)[1][0].detach().numpy()
        phrase_embedding=torch.tensor(phrase_embedding)
        predicted_labels = self.linear(phrase_embedding)
        predicted_labels=self.softmax(predicted_labels.unsqueeze(0))
        predicted_label=predicted_labels.argmax().item()
        proba = max(predicted_labels.detach().numpy()[0])
        label_name = next(key for key, value in bert.dic_lab.items() if value == predicted_label)
        return label_name , proba
    def validation_score(self):
        X_val=torch.tensor(self.X_val)
        Y_val=torch.tensor(self.Y_val)
        outputs = self.linear(X_val)
        outputs = self.softmax(outputs)
        predicted = outputs.argmax(1)
        correct = (predicted == Y_val).sum().item()
        accuracy = correct / len(Y_val)
        return accuracy    
    def create_csv(self,test_file,csv_file):
        csv_file = open(csv_file, 'w',newline='')
        writer = csv.writer(csv_file)
        writer.writerow(["ID","Label"])
        i=0
        with open(test_file, 'r') as file:
            for line in file:
                pred=self.predict(line)[0]
                writer.writerow([str(i), str(pred)])
                i+=1
        csv_file.close()


In [47]:
bert=BertClassifFinetuned('./data/train.json','./data/validation.json')

In [48]:
bert.train(epochs=30000)

Epoch [1/30000], Loss: 2.4841, Val score: 0.0833
Epoch [1001/30000], Loss: 2.4780, Val score: 0.1000
Epoch [2001/30000], Loss: 2.4688, Val score: 0.1167
Epoch [3001/30000], Loss: 2.4561, Val score: 0.1167
Epoch [4001/30000], Loss: 2.4460, Val score: 0.1167
Epoch [5001/30000], Loss: 2.4378, Val score: 0.1167
Epoch [6001/30000], Loss: 2.4301, Val score: 0.1167
Epoch [7001/30000], Loss: 2.4227, Val score: 0.1500
Epoch [8001/30000], Loss: 2.4154, Val score: 0.1500
Epoch [9001/30000], Loss: 2.4082, Val score: 0.1500
Epoch [10001/30000], Loss: 2.4011, Val score: 0.1667
Epoch [11001/30000], Loss: 2.3941, Val score: 0.1500
Epoch [12001/30000], Loss: 2.3873, Val score: 0.1500
Epoch [13001/30000], Loss: 2.3809, Val score: 0.1500
Epoch [14001/30000], Loss: 2.3748, Val score: 0.1333
Epoch [15001/30000], Loss: 2.3691, Val score: 0.1333
Epoch [16001/30000], Loss: 2.3638, Val score: 0.1333
Epoch [17001/30000], Loss: 2.3588, Val score: 0.1333
Epoch [18001/30000], Loss: 2.3541, Val score: 0.1167
Epoch 

In [49]:
bert.get_new_train(conf=0.9)

In [50]:
bert.data_augmentation()

In [51]:
bert.train(epochs=120000)

Epoch [1/120000], Loss: 2.1597, Val score: 0.1500
Epoch [1001/120000], Loss: 2.1534, Val score: 0.1500
Epoch [2001/120000], Loss: 2.1497, Val score: 0.1500
Epoch [3001/120000], Loss: 2.1462, Val score: 0.1500
Epoch [4001/120000], Loss: 2.1430, Val score: 0.1500
Epoch [5001/120000], Loss: 2.1400, Val score: 0.1500
Epoch [6001/120000], Loss: 2.1371, Val score: 0.1667
Epoch [7001/120000], Loss: 2.1344, Val score: 0.1667
Epoch [8001/120000], Loss: 2.1318, Val score: 0.1667
Epoch [9001/120000], Loss: 2.1294, Val score: 0.1667
Epoch [10001/120000], Loss: 2.1271, Val score: 0.1833
Epoch [11001/120000], Loss: 2.1248, Val score: 0.1833
Epoch [12001/120000], Loss: 2.1227, Val score: 0.1833
Epoch [13001/120000], Loss: 2.1207, Val score: 0.1833
Epoch [14001/120000], Loss: 2.1187, Val score: 0.1833
Epoch [15001/120000], Loss: 2.1168, Val score: 0.1667
Epoch [16001/120000], Loss: 2.1150, Val score: 0.1667
Epoch [17001/120000], Loss: 2.1133, Val score: 0.1667
Epoch [18001/120000], Loss: 2.1116, Val s

In [34]:
bert.create_csv('./data/test_shuffle.txt','./data/finetuned_bert.csv')

In [22]:
phrase="The Supreme Court has ruled in favor of the plaintiff in the high-profile case."

In [23]:
bert.predict(phrase)

('Politics', 0.99518234)

In [25]:
bert.save("./data/model.pth")

In [1]:
import json

In [68]:
train_file='./data/train.json'
validation_file='./data/validation.json'
test_file='./data/test_shuffle.txt'

In [3]:
list_sent=[]
list_lab=[]
with open(train_file, 'r') as file:
    data = file.read()
    data = json.loads(data)
    for label in data.keys():
        for sentence in data[label]:
            list_sent.append(sentence)
            list_lab.append(label)

In [6]:
Y_train=[]
dic_lab={}
for i in range(len(list_lab)):
    if list_lab[i] not in dic_lab:
        dic_lab[list_lab[i]]=len(dic_lab)
    Y_train.append(dic_lab[list_lab[i]])

In [8]:
dic_lab.keys()

dict_keys(['Politics', 'Health', 'Finance', 'Travel', 'Food', 'Education', 'Environment', 'Fashion', 'Science', 'Sports', 'Technology', 'Entertainment'])

In [18]:
labels= [key.lower() for key in dic_lab.keys()]

In [28]:
new_dic={}
for i in range(len(labels)):
    new_dic[labels[i].lower()]=i


In [39]:
k=0
i=0
with open(test_file, 'r') as file:
    for line in file:
        for words in line.split(" "):
            if words.lower() in labels:
                k+=1 
        i+=1

PREDICT


In [83]:
import torch
import torch.nn as nn
import json
from transformers import BertTokenizer, BertModel
import random
import csv

class pipeline():
    def __init__(self,train_file,validation_file):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertModel.from_pretrained("bert-base-uncased")
        self.linear = torch.nn.Linear(768,12,bias=True)
        self.softmax = torch.nn.Softmax(dim=1)
        self.train_file=train_file
        list_sent=[]
        list_lab=[]
        with open(train_file, 'r') as file:
            data = file.read()
            data = json.loads(data)
            for label in data.keys():
                for sentence in data[label]:
                    list_sent.append(sentence)
                    list_lab.append(label)
        self.list_lab=list_lab
        self.list_sent=list_sent
        Y_train=[]
        self.dic_lab={}
        for i in range(len(list_lab)):
            if list_lab[i] not in self.dic_lab:
                self.dic_lab[list_lab[i]]=len(self.dic_lab)
            Y_train.append(self.dic_lab[list_lab[i]])
        X_train=[]
        for sent in list_sent:
            encoded_input = self.tokenizer(sent, return_tensors='pt')
            output = self.model(**encoded_input)
            X_train.append(output[1][0].detach().numpy())
        self.X_train=X_train
        self.Y_train=Y_train
        self.sent_train=list_sent
        self.lab_train=list_lab

        list_sent_val=[]
        list_lab_val=[]
        with open(validation_file, 'r') as file:
            data = file.read()
            data = json.loads(data)
            for label in data.keys():
                for sentence in data[label]:
                    list_sent_val.append(sentence)
                    list_lab_val.append(label)
        self.lab_val=list_lab_val
        self.sent_val=list_sent_val
        Y_val=[]
        X_val=[]
        for i,sent in enumerate(list_sent_val):
            encoded_input = self.tokenizer(sent, return_tensors='pt')
            output = self.model(**encoded_input)
            X_val.append(output[1][0].detach().numpy())
            Y_val.append(self.dic_lab[list_lab_val[i]])
        self.X_val=X_val
        self.Y_val=Y_val

        self.labels= [key.lower() for key in self.dic_lab.keys()]
        for key in dic_lab.keys():
            new_dic[key.lower()]=key 
        self.lower_dic = new_dic
    def train(self,epochs=10000):
        X_train=torch.tensor(self.X_train)
        Y_train=torch.tensor(self.Y_train)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.SGD(self.linear.parameters(), lr=0.001)

        train_data = list(zip(X_train, Y_train))
        random.shuffle(train_data)
        X_train, Y_train = zip(*train_data)
        X_train = torch.stack(X_train)
        Y_train = torch.tensor(Y_train)

        for epoch in range(epochs):
            optimizer.zero_grad()
            outputs = self.linear(X_train)
            outputs = self.softmax(outputs)
            loss = criterion(outputs, Y_train)
            loss.backward()
            optimizer.step()
            if epoch % 1000 == 0:
                print ('Epoch [{}/{}], Loss: {:.4f}, Val score: {:.4f}'.format(epoch+1, epochs, loss.item(), self.validation_score()))
    def data_augmentation(self):
        "On augmente le jeu de données en remplaçant un mot de chaque phrase par [MASK]"
        list_sent = self.sent_train
        new_sent=[]
        list_lab = self.lab_train
        for phrase in list_sent:
            words=phrase.split(" ")
            i=random.randint(0,len(words)-1)
            words[i]="[MASK]"
            phrase=" ".join(words)
            new_sent.append(phrase)
        X_train=[]
        for sent in list_sent:
            encoded_input = self.tokenizer(sent, return_tensors='pt')
            output = self.model(**encoded_input)
            X_train.append(output[1][0].detach().numpy())
        self.X_train = self.X_train + X_train
        self.Y_train = self.Y_train + self.Y_train

    def get_new_train(self,conf=0.99,method=1,test_file='./data/test_shuffle.txt'):
        "On utilise du semi-supervised learning pour ajouter des données au jeu d'entrainement"
        list_sent=[]
        list_lab=[]
        data=[]
        with open(test_file, 'r') as file:
            for line in file:
                if method==1:
                    pred=self.predict(line)
                if method==2:
                    pred=self.predict_v2(line)
                else:
                    raise ValueError("This method does not exist")
                data.append([pred[1],pred[0],line])
        for x in data:
            if x[0]>conf and x[2] not in self.sent_train:
                list_sent.append(x[2])
                list_lab.append(x[1])
        X_train=[]
        Y_train=[]
        for i,sent in enumerate(list_sent):
            encoded_input = self.tokenizer(sent, return_tensors='pt')
            output = self.model(**encoded_input)
            X_train.append(output[1][0].detach().numpy())
            Y_train.append(self.dic_lab[list_lab[i]])
        self.X_train = self.X_train + X_train
        self.Y_train = self.Y_train + Y_train
        self.sent_train = self.sent_train + list_sent
        self.lab_train = self.lab_train + list_lab
    def save(self,path='model.pth'):
        torch.save(self.linear.state_dict(), path)
    def load(self,path='model.pth'):
        self.linear.load_state_dict(torch.load(path))
    def predict(self,phrase):
        encoded_phrase = self.tokenizer(phrase, return_tensors='pt')
        phrase_embedding = self.model(**encoded_phrase)[1][0].detach().numpy()
        phrase_embedding=torch.tensor(phrase_embedding)
        predicted_labels = self.linear(phrase_embedding)
        predicted_labels=self.softmax(predicted_labels.unsqueeze(0))
        predicted_label=predicted_labels.argmax().item()
        proba = max(predicted_labels.detach().numpy()[0])
        label_name = next(key for key, value in self.dic_lab.items() if value == predicted_label)
        return label_name , proba
    def predict_v2(self,phrase):
        for words in phrase.split(" "):
            if words.lower() in self.labels:
                return self.lower_dic[words.lower()], 0.95
        encoded_phrase = self.tokenizer(phrase, return_tensors='pt')
        phrase_embedding = self.model(**encoded_phrase)[1][0].detach().numpy()
        phrase_embedding=torch.tensor(phrase_embedding)
        predicted_labels = self.linear(phrase_embedding)
        predicted_labels=self.softmax(predicted_labels.unsqueeze(0))
        predicted_label=predicted_labels.argmax().item()
        proba = max(predicted_labels.detach().numpy()[0])
        label_name = next(key for key, value in self.dic_lab.items() if value == predicted_label)
        return label_name , proba
    def validation_score(self):
        X_val=torch.tensor(self.X_val)
        Y_val=torch.tensor(self.Y_val)
        outputs = self.linear(X_val)
        outputs = self.softmax(outputs)
        predicted = outputs.argmax(1)
        correct = (predicted == Y_val).sum().item()
        accuracy = correct / len(Y_val)
        return accuracy    
    def create_csv(self,test_file,csv_file):
        csv_file = open(csv_file, 'w',newline='')
        writer = csv.writer(csv_file)
        writer.writerow(["ID","Label"])
        new_dic={}
        labels= [key.lower() for key in self.dic_lab.keys()]
        for key in dic_lab.keys():
            new_dic[key.lower()]=key 
        i=0
        with open(test_file, 'r') as file:
            for line in file:
                predicted = False
                for words in line.split(" "):
                    if words.lower() in labels:
                        pred=new_dic[words.lower()]
                        predicted = True
                if not predicted:
                    pred=self.predict(line)[0]
                writer.writerow([str(i), str(pred)])
                i+=1
        csv_file.close()


In [88]:
pipe = pipeline('./data/train.json','./data/validation.json')

In [89]:
pipe.data_augmentation()

In [90]:
pipe.train(epochs=30000)

Epoch [1/30000], Loss: 2.4849, Val score: 0.1500
Epoch [1001/30000], Loss: 2.4792, Val score: 0.1500
Epoch [2001/30000], Loss: 2.4719, Val score: 0.1333
Epoch [3001/30000], Loss: 2.4608, Val score: 0.1167
Epoch [4001/30000], Loss: 2.4501, Val score: 0.1333
Epoch [5001/30000], Loss: 2.4420, Val score: 0.1333
Epoch [6001/30000], Loss: 2.4353, Val score: 0.1333
Epoch [7001/30000], Loss: 2.4292, Val score: 0.1333
Epoch [8001/30000], Loss: 2.4233, Val score: 0.1333
Epoch [9001/30000], Loss: 2.4173, Val score: 0.1333
Epoch [10001/30000], Loss: 2.4111, Val score: 0.1333
Epoch [11001/30000], Loss: 2.4044, Val score: 0.1333
Epoch [12001/30000], Loss: 2.3972, Val score: 0.1500
Epoch [13001/30000], Loss: 2.3897, Val score: 0.1667
Epoch [14001/30000], Loss: 2.3822, Val score: 0.1667
Epoch [15001/30000], Loss: 2.3749, Val score: 0.1667
Epoch [16001/30000], Loss: 2.3678, Val score: 0.1667
Epoch [17001/30000], Loss: 2.3611, Val score: 0.1667
Epoch [18001/30000], Loss: 2.3547, Val score: 0.1667
Epoch 

In [98]:
pipe.get_new_train(conf=0.9,method=2)

In [99]:
pipe.data_augmentation()

In [100]:
pipe.train(epochs=120000)

Epoch [1/120000], Loss: 2.3618, Val score: 0.2000
Epoch [1001/120000], Loss: 2.3616, Val score: 0.1833
Epoch [2001/120000], Loss: 2.3615, Val score: 0.1833
Epoch [3001/120000], Loss: 2.3615, Val score: 0.1833
Epoch [4001/120000], Loss: 2.3614, Val score: 0.1833
Epoch [5001/120000], Loss: 2.3614, Val score: 0.1833
Epoch [6001/120000], Loss: 2.3613, Val score: 0.1833
Epoch [7001/120000], Loss: 2.3612, Val score: 0.1833
Epoch [8001/120000], Loss: 2.3612, Val score: 0.1833
Epoch [9001/120000], Loss: 2.3611, Val score: 0.1833
Epoch [10001/120000], Loss: 2.3611, Val score: 0.1833
Epoch [11001/120000], Loss: 2.3610, Val score: 0.1833
Epoch [12001/120000], Loss: 2.3610, Val score: 0.1833
Epoch [13001/120000], Loss: 2.3609, Val score: 0.1833
Epoch [14001/120000], Loss: 2.3609, Val score: 0.1833
Epoch [15001/120000], Loss: 2.3608, Val score: 0.1833
Epoch [16001/120000], Loss: 2.3608, Val score: 0.1833
Epoch [17001/120000], Loss: 2.3607, Val score: 0.1833
Epoch [18001/120000], Loss: 2.3607, Val s

In [101]:
pipe.save("./data/model3.pth")

In [102]:
pipe.load("./data/model2.pth")

In [103]:
pipe.create_csv('./data/test_shuffle.txt','./data/pipeline2.csv')

Sentence Transformers

In [3]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")

# Our sentences to encode
sentences = [
    "This framework generates embeddings for each input sentence",
    "Sentences are passed as a list of string.",
    "The quick brown fox jumps over the lazy dog."
]

# Sentences are encoded by calling model.encode()
embeddings = model.encode(sentences)

# Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print(len(embedding))


384
384
384


In [11]:
from sentence_transformers import SentenceTransformer
from sklearn.neighbors import KNeighborsClassifier
import json
import csv

class SentenceTransFeature():
    def __init__(self):
        self.model = SentenceTransformer("all-MiniLM-L6-v2")
        self.knn = KNeighborsClassifier(n_neighbors=1)
    def train(self,train_file):
        list_sent=[]
        Y_train=[]
        with open(train_file, 'r') as file:
            data = file.read()
            data = json.loads(data)
            for label in data.keys():
                for sentence in data[label]:
                    list_sent.append(sentence)
                    Y_train.append(label)
        X_train=[]
        for sent in list_sent:
            output = self.model.encode(sent)
            X_train.append(output)#[1][0].detach().numpy())
        self.knn.fit(X_train, Y_train)
    def predict(self,phrase):
        phrase_embedding = self.model.encode(phrase)#[1][0].detach().numpy()
        predicted_label = self.knn.predict([phrase_embedding])
        return predicted_label
    def create_csv(self,test_file,csv_file):
        csv_file = open(csv_file, 'w',newline='')
        writer = csv.writer(csv_file)
        writer.writerow(["ID","Label"])
        i=0
        with open(test_file, 'r') as file:
            for line in file:
                pred=self.predict(line)[0]
                writer.writerow([str(i), str(pred)])
                i+=1
        csv_file.close()

In [12]:
model=SentenceTransFeature()

In [13]:
model.train('./data/train.json')

In [14]:
model.create_csv('./data/test_shuffle.txt','./data/feature_based_sentence.csv')

In [59]:
import torch
import torch.nn as nn
import json
from sentence_transformers import SentenceTransformer
import random
import csv

class pipeline_sent():
    def __init__(self,train_file,validation_file):
        self.model = SentenceTransformer("all-MiniLM-L6-v2")
        self.linear = torch.nn.Linear(384,12,bias=True)
        self.softmax = torch.nn.Softmax(dim=1)
        self.train_file=train_file
        list_sent=[]
        list_lab=[]
        with open(train_file, 'r') as file:
            data = file.read()
            data = json.loads(data)
            for label in data.keys():
                for sentence in data[label]:
                    list_sent.append(sentence)
                    list_lab.append(label)
        self.list_lab=list_lab
        self.list_sent=list_sent
        Y_train=[]
        self.dic_lab={}
        for i in range(len(list_lab)):
            if list_lab[i] not in self.dic_lab:
                self.dic_lab[list_lab[i]]=len(self.dic_lab)
            Y_train.append(self.dic_lab[list_lab[i]])
        X_train=[]
        for sent in list_sent:
            output = self.model.encode(sent)
            X_train.append(output)
        self.X_train=X_train
        self.Y_train=Y_train
        self.sent_train=list_sent
        self.lab_train=list_lab

        list_sent_val=[]
        list_lab_val=[]
        with open(validation_file, 'r') as file:
            data = file.read()
            data = json.loads(data)
            for label in data.keys():
                for sentence in data[label]:
                    list_sent_val.append(sentence)
                    list_lab_val.append(label)
        self.lab_val=list_lab_val
        self.sent_val=list_sent_val
        Y_val=[]
        X_val=[]
        for i,sent in enumerate(list_sent_val):
            output = self.model.encode(sent)
            X_val.append(output)
            Y_val.append(self.dic_lab[list_lab_val[i]])
        self.X_val=X_val
        self.Y_val=Y_val
        new_dic={}
        self.labels= [key.lower() for key in self.dic_lab.keys()]
        for key in self.dic_lab.keys():
            new_dic[key.lower()]=key 
        self.lower_dic = new_dic
    def train(self,epochs=10000):
        X_train=torch.tensor(self.X_train)
        Y_train=torch.tensor(self.Y_train)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.SGD(self.linear.parameters(), lr=0.001)

        train_data = list(zip(X_train, Y_train))
        random.shuffle(train_data)
        X_train, Y_train = zip(*train_data)
        X_train = torch.stack(X_train)
        Y_train = torch.tensor(Y_train)

        for epoch in range(epochs):
            optimizer.zero_grad()
            outputs = self.linear(X_train)
            outputs = self.softmax(outputs)
            loss = criterion(outputs, Y_train)
            loss.backward()
            optimizer.step()
            if epoch % 1000 == 0:
                print ('Epoch [{}/{}], Loss: {:.4f}, Val score: {:.4f}'.format(epoch+1, epochs, loss.item(), self.validation_score()))
    def data_augmentation(self):
        "On augmente le jeu de données en remplaçant un mot de chaque phrase par [MASK]"
        list_sent = self.sent_train
        new_sent=[]
        list_lab = self.lab_train
        for phrase in list_sent:
            words=phrase.split(" ")
            i=random.randint(0,len(words)-1)
            words[i]="[MASK]"
            phrase=" ".join(words)
            new_sent.append(phrase)
        X_train=[]
        for sent in list_sent:
            output = self.model.encode(sent)
            X_train.append(output)
        self.X_train = self.X_train + X_train
        self.Y_train = self.Y_train + self.Y_train

    def get_new_train(self,conf=0.99,method=1,test_file='./data/test_shuffle.txt'):
        "On utilise du semi-supervised learning pour ajouter des données au jeu d'entrainement"
        list_sent=[]
        list_lab=[]
        data=[]
        with open(test_file, 'r') as file:
            for line in file:
                if method==1:
                    pred=self.predict(line)
                elif method==2:
                    pred=self.predict_v2(line)
                else:
                    raise ValueError("This method does not exist")
                data.append([pred[1],pred[0],line])
        for x in data:
            if x[0]>conf and x[2] not in self.sent_train:
                list_sent.append(x[2])
                list_lab.append(x[1])
        X_train=[]
        Y_train=[]
        for i,sent in enumerate(list_sent):
            output = self.model.encode(sent)
            X_train.append(output)
            Y_train.append(self.dic_lab[list_lab[i]])
        self.X_train = self.X_train + X_train
        self.Y_train = self.Y_train + Y_train
        self.sent_train = self.sent_train + list_sent
        self.lab_train = self.lab_train + list_lab
    def save(self,path='model.pth'):
        torch.save(self.linear.state_dict(), path)
    def load(self,path='model.pth'):
        self.linear.load_state_dict(torch.load(path))
    def predict(self,phrase):
        phrase_embedding = self.model.encode(phrase)
        phrase_embedding=torch.tensor(phrase_embedding)
        predicted_labels = self.linear(phrase_embedding)
        predicted_labels=self.softmax(predicted_labels.unsqueeze(0))
        predicted_label=predicted_labels.argmax().item()
        proba = max(predicted_labels.detach().numpy()[0])
        label_name = next(key for key, value in self.dic_lab.items() if value == predicted_label)
        return label_name , proba
    def predict_v2(self,phrase):
        for words in phrase.split(" "):
            if words.lower() in self.labels:
                return self.lower_dic[words.lower()], 0.95
        phrase_embedding = self.model.encode(phrase)
        phrase_embedding=torch.tensor(phrase_embedding)
        predicted_labels = self.linear(phrase_embedding)
        predicted_labels=self.softmax(predicted_labels.unsqueeze(0))
        predicted_label=predicted_labels.argmax().item()
        proba = max(predicted_labels.detach().numpy()[0])
        label_name = next(key for key, value in self.dic_lab.items() if value == predicted_label)
        return label_name , proba
    def validation_score(self):
        X_val=torch.tensor(self.X_val)
        Y_val=torch.tensor(self.Y_val)
        outputs = self.linear(X_val)
        outputs = self.softmax(outputs)
        predicted = outputs.argmax(1)
        correct = (predicted == Y_val).sum().item()
        accuracy = correct / len(Y_val)
        return accuracy    
    def create_csv(self,test_file,csv_file):
        csv_file = open(csv_file, 'w',newline='')
        writer = csv.writer(csv_file)
        writer.writerow(["ID","Label"])
        new_dic={}
        labels= [key.lower() for key in self.dic_lab.keys()]
        for key in self.dic_lab.keys():
            new_dic[key.lower()]=key 
        i=0
        with open(test_file, 'r') as file:
            for line in file:
                predicted = False
                for words in line.split(" "):
                    if words.lower() in labels:
                        pred=new_dic[words.lower()]
                        predicted = True
                if not predicted:
                    pred=self.predict(line)[0]
                writer.writerow([str(i), str(pred)])
                i+=1
        csv_file.close()


In [60]:
model=pipeline_sent('./data/train.json','./data/validation.json')

In [61]:
model.load("./data/model_sent.pth")

In [62]:
model.create_csv('./data/test_shuffle.txt','./data/pipeline_sent.csv')

In [46]:
model.data_augmentation()

In [39]:
model.train(epochs=120000)

Epoch [1/120000], Loss: 2.4843, Val score: 0.1333
Epoch [1001/120000], Loss: 2.4841, Val score: 0.1333
Epoch [2001/120000], Loss: 2.4838, Val score: 0.1333
Epoch [3001/120000], Loss: 2.4836, Val score: 0.1500
Epoch [4001/120000], Loss: 2.4834, Val score: 0.1500
Epoch [5001/120000], Loss: 2.4832, Val score: 0.1500
Epoch [6001/120000], Loss: 2.4830, Val score: 0.1333
Epoch [7001/120000], Loss: 2.4828, Val score: 0.1333
Epoch [8001/120000], Loss: 2.4826, Val score: 0.1333
Epoch [9001/120000], Loss: 2.4823, Val score: 0.1333
Epoch [10001/120000], Loss: 2.4821, Val score: 0.1667
Epoch [11001/120000], Loss: 2.4819, Val score: 0.1833
Epoch [12001/120000], Loss: 2.4817, Val score: 0.2000
Epoch [13001/120000], Loss: 2.4815, Val score: 0.2000
Epoch [14001/120000], Loss: 2.4812, Val score: 0.2000
Epoch [15001/120000], Loss: 2.4810, Val score: 0.2000
Epoch [16001/120000], Loss: 2.4808, Val score: 0.2167
Epoch [17001/120000], Loss: 2.4806, Val score: 0.2167
Epoch [18001/120000], Loss: 2.4803, Val s

In [54]:
model.get_new_train(conf=0.99,method=1)

In [55]:
model.data_augmentation()

In [56]:
model.train(epochs=30000)

Epoch [1/30000], Loss: 2.4368, Val score: 0.5667
Epoch [1001/30000], Loss: 2.4363, Val score: 0.5667
Epoch [2001/30000], Loss: 2.4358, Val score: 0.5500
Epoch [3001/30000], Loss: 2.4353, Val score: 0.5500
Epoch [4001/30000], Loss: 2.4348, Val score: 0.5667
Epoch [5001/30000], Loss: 2.4343, Val score: 0.5667
Epoch [6001/30000], Loss: 2.4338, Val score: 0.5667
Epoch [7001/30000], Loss: 2.4333, Val score: 0.5667
Epoch [8001/30000], Loss: 2.4328, Val score: 0.5667
Epoch [9001/30000], Loss: 2.4323, Val score: 0.5667
Epoch [10001/30000], Loss: 2.4318, Val score: 0.5667
Epoch [11001/30000], Loss: 2.4313, Val score: 0.5667
Epoch [12001/30000], Loss: 2.4308, Val score: 0.5667
Epoch [13001/30000], Loss: 2.4302, Val score: 0.5667
Epoch [14001/30000], Loss: 2.4297, Val score: 0.5667
Epoch [15001/30000], Loss: 2.4292, Val score: 0.5667
Epoch [16001/30000], Loss: 2.4287, Val score: 0.5667
Epoch [17001/30000], Loss: 2.4281, Val score: 0.5500
Epoch [18001/30000], Loss: 2.4276, Val score: 0.5500
Epoch 

In [41]:
model.save("./data/model_sent.pth")